# Import Library

In [1]:
import random
import json
import pickle
import numpy as np
import nltk
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD
import tensorflow as tf


# Tokenization

In [2]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.download('punkt')
nltk.download('wordnet')
intents=json.loads(open('C://Users//GMY//Music//chatbot//chatbot.json',encoding="utf8").read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words=sorted(list(set(words)))
word=set(words[0])

classes=sorted(list(set(classes)))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GMY\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\GMY\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Normalization

In [3]:

def normalize_char_level_missmatch(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ  
    rep49=re.sub('[ሐ]','ሀ',rep48)
    rep50=re.sub('[ሑ]','ሁ',rep49)
    rep51=re.sub('[ሒ]','ሂ',rep50)
    rep52=re.sub('[ሓ]','ሃ',rep51)
    rep53=re.sub('[ሔ]','ሄ',rep52)
    rep54=re.sub('[ሕ]','ህ',rep53)
    rep55=re.sub('[ሖ]','ሆ',rep54)
    rep56=re.sub('[?]','',rep55)
    return rep56
    


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# Preprocessing

In [9]:
training=[]
output_empty=[0]*len(classes)

for document in documents:
     bag=[]
     pattern_words=document[0]
     
     for word in words:
        bag.append(1) if word in pattern_words else bag.append(0)
        
     output_row=list(output_empty)
     
     output_row[classes.index(document[1])]=1
     
     training.append([bag,output_row])
     
random.shuffle(training)

training=np.array(training)
train_x=list(training[:,0])

train_y=list(training[:,1])


C:\Users\GMY\AppData\Local\Temp\ipykernel_12264\2411667134.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


# Model Training

In [10]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=tf.keras.optimizers.legacy.SGD(learning_rate=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
his=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=0)

# Query 

In [15]:
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentences):
   sentence_words=nltk.word_tokenize(sentences)
   return sentence_words

In [16]:
ERROR_THRESHOLD=0.3
def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    
    for w in sentence_words:
        for i,word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)

# Model Prediction

In [17]:
def predict_class(sentence):
    bow=bag_of_words(sentence)
    res=model.predict(np.array([bow]))[0]
    error=0.25
    result=[[i,r] for i,r in enumerate(res) if r>error]
    result.sort(key=lambda x:x[1],reverse=True)
    result_list=[]
    for r in result:
        result_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return result_list

# Get Response

In [18]:
def get_response(intents_list,intents_json):
    tag=intents_list[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break;
        else:
          result="none"
    return result

# GUI

In [23]:
import tkinter
from tkinter import *
base = Tk()
base.title("MentalHealthCare ChatBot")#by default
base.geometry("1150x600")
base.resizable(width=FALSE, height=FALSE)
base.iconbitmap("C://Users//GMY//Downloads//AI DOCUMENTATION//images.ico")
   

''

In [20]:
# Send Function

In [24]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "እርሶ： " + msg + '\n' +'\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        mk=normalize_char_level_missmatch(msg)

        ints=predict_class(mk)
        res=get_response(ints,intents)
        ChatLogout.config(state=NORMAL)
        ChatLogout.insert(END, '\n'  +"ቴራፒስት： " + res + '\n' +'\n' )
        ChatLogout.config(foreground="#007cc7", font=("Verdana", 12 ))
        ChatLogout.config(state=DISABLED)
        ChatLogout.yview(END)
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [ ]:
icon = PhotoImage(file='C://Users//GMY//Pictures//icons//sendicon.png')

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
ChatLogout = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLogout.config(state=DISABLED)


scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", image=icon,width=70,height=60,borderwidth=0,
                     
                     command= send )
EntryBox = Text(base, bd=0,bg="#E0E0E0",width="29", height="5", font="Arial")
scrollbar.place(x=1130,y=6, height=600)
ChatLog.place(x=6,y=6, height=500, width=430)
ChatLogout.place(x=430,y=6, height=500, width=700)

EntryBox.place(x=6, y=510, height=60, width=700)
SendButton.place(x=735, y=510, height=75 )
base.mainloop()